In [1]:
%%capture
!pip install sklearn
!pip install pandas
!pip install numpy

You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [6]:
games = pd.read_csv("data/games_7_players.csv")

## Define X and y

In [16]:
x_range = range(14,392)

In [17]:
x_range

range(14, 392)

In [18]:
X = games.iloc[:, x_range]

In [19]:
X

,fgm_seas_avg_p1,fgm_seas_avg_p2,fgm_seas_avg_p3,fgm_seas_avg_p4,fgm_seas_avg_p5,fgm_seas_avg_p6,fgm_seas_avg_p7,fgm_seas_avg_p8,fgm_seas_avg_p9,fgm_seas_avg_p10,...,pf_l5_p5,pf_l5_p6,pf_l5_p7,pf_l5_p8,pf_l5_p9,pf_l5_p10,pf_l5_p11,pf_l5_p12,pf_l5_p13,pf_l5_p14
0,6.067568,4.656716,7.848101,5.548780,6.057971,5.531646,2.000000,7.287671,6.444444,3.632911,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,9.756098,4.844156,4.397260,7.308642,0.000000,4.435897,5.620000,1.641791,3.560976,1.612500,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.000000,7.088608,2.424242,6.357143,6.050633,3.701299,7.350000,9.700000,3.901235,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,4.575758,7.987013,6.316456,2.978261,5.493506,4.707317,5.525641,3.631579,3.074074,2.618421,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,7.088608,2.424242,6.357143,6.050633,3.701299,7.350000,7.086420,5.408163,0.000000,6.531646,...,2.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12600,6.481481,10.145455,9.464286,5.765625,3.804348,5.333333,0.000000,3.186047,7.085714,4.777778,...,1.0,1.8,3.2,2.2,2.800000,3.8,4.2,2.6,2.8,2.6
12601,7.700000,6.840000,3.975000,5.137931,4.600000,0.000000,6.142857,3.488372,0.000000,4.219512,...,3.2,1.2,2.6,2.6,1.400000,0.6,1.4,1.8,2.8,0.8
12602,8.164179,5.947368,0.000000,2.848485,2.326087,0.000000,3.000000,4.830189,0.000000,0.000000,...,2.8,1.5,2.6,2.4,1.333333,2.2,3.0,1.6,1.0,3.6
12603,9.317073,8.762500,6.025641,5.777778,4.291667,2.684211,1.651163,2.113208,5.432099,2.833333,...,3.6,2.2,4.8,1.6,2.000000,1.6,1.4,2.0,2.6,2.2


In [20]:
y = games.iloc[:, 406:420]

In [21]:
y

,fp_p1,fp_p2,fp_p3,fp_p4,fp_p5,fp_p6,fp_p7,fp_p8,fp_p9,fp_p10,fp_p11,fp_p12,fp_p13,fp_p14
0,69.2,62.8,59.1,52.4,49.5,31.9,30.7,27.8,25.4,24.4,16.9,14.2,12.2,11.6
1,65.5,61.7,47.7,45.2,43.7,43.5,34.3,25.5,19.1,15.5,10.2,10.1,9.9,7.9
2,51.9,48.0,38.7,38.6,37.9,34.7,34.4,33.5,30.5,23.3,21.1,18.2,16.7,14.2
3,45.2,43.3,39.4,36.5,35.9,35.9,34.3,33.5,29.8,24.0,14.5,14.1,12.4,11.2
4,30.9,32.8,29.8,20.0,45.3,20.6,47.7,39.8,36.4,27.8,22.8,22.7,20.1,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12600,55.1,77.4,37.7,37.1,28.9,22.4,21.9,16.2,33.5,23.3,21.3,29.1,40.4,11.8
12601,29.0,81.8,28.5,38.0,47.1,26.4,32.0,19.8,30.5,21.4,31.3,27.8,30.9,32.6
12602,51.1,46.1,45.4,31.2,26.0,44.0,19.3,22.4,36.5,36.4,27.3,26.2,27.4,34.0
12603,66.2,68.4,28.8,52.7,32.2,42.1,24.1,37.5,29.3,26.5,27.1,32.6,16.2,22.3


## Splitting data


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.10)

In [23]:
model = keras.models.Sequential([
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(14, activation='relu'),
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [24]:
m = model.fit(X_train, y_train, validation_split=0.20, epochs=100)

print(model.evaluate(X_test,  y_test, verbose=2))

model.metrics_names

Train on 9075 samples, validate on 2269 samples
Epoch 1/100
9075/9075 [==============================] - 2s 195us/sample - loss: 244.0995 - mae: 11.3574 - val_loss: 179.8726 - val_mae: 10.0409
Epoch 2/100
9075/9075 [==============================] - 1s 91us/sample - loss: 180.1248 - mae: 9.9847 - val_loss: 177.2729 - val_mae: 9.8895
Epoch 3/100
9075/9075 [==============================] - 1s 91us/sample - loss: 177.7027 - mae: 9.8999 - val_loss: 174.8736 - val_mae: 9.8171
Epoch 4/100
9075/9075 [==============================] - 1s 90us/sample - loss: 156.4784 - mae: 9.3898 - val_loss: 112.8981 - val_mae: 8.3359
Epoch 5/100
9075/9075 [==============================] - 1s 90us/sample - loss: 111.9105 - mae: 8.2903 - val_loss: 111.1517 - val_mae: 8.3353
Epoch 6/100
9075/9075 [==============================] - 1s 90us/sample - loss: 110.9244 - mae: 8.2542 - val_loss: 110.5895 - val_mae: 8.2900
Epoch 7/100
9075/9075 [==============================] - 1s 91us/sample - loss: 109.9982 - mae: 8

['loss', 'mae']

In [215]:
games.iloc[12603:12604, 14:406]

,fgm_seas_avg_p1,fgm_seas_avg_p2,fgm_seas_avg_p3,fgm_seas_avg_p4,fgm_seas_avg_p5,fgm_seas_avg_p6,fgm_seas_avg_p7,fgm_seas_avg_p8,fgm_seas_avg_p9,fgm_seas_avg_p10,...,fp_l5_p5,fp_l5_p6,fp_l5_p7,fp_l5_p8,fp_l5_p9,fp_l5_p10,fp_l5_p11,fp_l5_p12,fp_l5_p13,fp_l5_p14
12603,9.317073,8.7625,6.025641,5.777778,4.291667,2.684211,1.651163,2.113208,5.432099,2.833333,...,33.48,32.8,32.54,31.92,27.36,26.54,26.54,24.72,24.44,24.34


In [216]:
new_X = games.iloc[12601:12604, 14:406].values

In [217]:
pd.DataFrame(model.predict(new_X))

ValueError: Input 0 of layer sequential_28 is incompatible with the layer: expected axis -1 of input shape to have value 14 but received input with shape [None, 392]

In [201]:
games.iloc[12601:12604, 406:419]

,fp_p1,fp_p2,fp_p3,fp_p4,fp_p5,fp_p6,fp_p7,fp_p8,fp_p9,fp_p10,fp_p11,fp_p12,fp_p13
12601,29.0,81.8,28.5,38.0,47.1,26.4,32.0,19.8,30.5,21.4,31.3,27.8,30.9
12602,51.1,46.1,45.4,31.2,26.0,44.0,19.3,22.4,36.5,36.4,27.3,26.2,27.4
12603,66.2,68.4,28.8,52.7,32.2,42.1,24.1,37.5,29.3,26.5,27.1,32.6,16.2
